In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw05.ipynb")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
from pathlib import Path

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
%matplotlib inline

In [3]:
# Setup - Load the SQL extension and connect to the Mini IMDB dataset we've prepared
db_path = Path('data/imdbmini.db')

engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = sqlalchemy.inspect(engine)

#Get table names
inspector.get_table_names()



['Name', 'Rating', 'Role', 'Title']

In [4]:
query_name = """
SELECT * 
FROM Name
"""
pd.read_sql(query_name, engine).head()

,nconst,primaryName,birthYear,deathYear,primaryProfession
0,1,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous"
1,2,Lauren Bacall,1924,2014,"actress,soundtrack"
2,3,Brigitte Bardot,1934,None,"actress,soundtrack,music_department"
3,4,John Belushi,1949,1982,"actor,soundtrack,writer"
4,5,Ingmar Bergman,1918,2007,"writer,director,actor"


In [5]:
query_role = """
SELECT * 
FROM Rating
"""
pd.read_sql(query_role, engine).head()

,index,tconst,averageRating,numVotes
0,0,417,8.2,46382
1,1,4972,6.3,23836
2,2,10323,8.1,60244
3,3,12349,8.3,120059
4,4,13442,7.9,92218


In [ ]:
query_rating = """
SELECT * 
FROM Role
"""
pd.read_sql(query_rating, engine).head()

,index,tconst,ordering,nconst,category,job,characters
0,0,417,1,617588,actor,None,"[""Prof. Barbenfouillis"",""The Moon""]"
1,1,417,2,29244,actor,None,"[""Astronomer""]"
2,2,417,3,76933,actress,None,"[""Lady in the Moon""]"
3,3,417,4,1215996,actor,None,"[""Astronomer""]"
4,4,417,5,894523,writer,"novel ""De la Terre à la Lune""",None


In [ ]:
query_title = """
SELECT * 
FROM Title 
LIMIT 3
"""
pd.read_sql(query_title, engine).head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,417,short,A Trip to the Moon,Le voyage dans la lune,0,1902,None,13,"Action,Adventure,Comedy"
1,4972,movie,The Birth of a Nation,The Birth of a Nation,0,1915,None,195,"Drama,History,War"
2,10323,movie,The Cabinet of Dr. Caligari,Das Cabinet des Dr. Caligari,0,1920,None,76,"Fantasy,Horror,Mystery"


## Question 1: Find all *distinct* film roles.

<br/>
<details>
    <summary>▶ [Click to Expand] <b>Hints + Output</b></summary>

- Tests: `DISTINCT`
- Our solution is 2 lines.
- **Hint Data**: The table `Role` contains all film roles.
- **Hint**: Use `DISTINCT`
</details>

The first 3 lines of the table should look like the following (but you should compute the entire table).
    
|category|
|---|
|actor|
|actress |
|writer |

<!--
BEGIN QUESTION
name: q1
points: 1
-->

In [14]:
query_q1 = """
SELECT DISTINCT category
FROM Role

"""


res_q1 = pd.read_sql(query_q1, engine)
res_q1.head()

,category
0,actor
1,actress
2,writer
3,composer
4,editor


In [ ]:
grader.check("q1")

## Question 2: Compute number of film releases, annually.

Details: Sort from newest to oldest, and ignore titles with `NULL` years.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>

- Tests: grouping, aggregation, handling NULL
- Our solution is 7 lines.
    
- **Approach**: You need to (#1) filter out `NULL` years, (#2) group by year and (#3) count within each group.
        
- **Hint #1**: For filtering out `NULL` values, see lecture 9.5 @ 2:20 https://youtu.be/LEMkOxv599U?t=140
    
- **Hint #2**: For using `GROUP BY`, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166
    
- **Hint Data**: Per the IMDB Dataset description, use the column `startYear` to determine when a film was released.
    
- **Hint**: For changing `ORDER BY` direction, consider `DESC` (descending) and `ASC` (ascending).
</details>

Output the year `year` and the number of films per year `total`. The first 3 lines of the table should look like the following (but you should compute the first 10 rows).
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>year</th>
      <th>total</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2021</td>
      <td>128</td>
    </tr>
    <tr>
      <td>2020</td>
      <td>181</td>
    </tr>
    <tr>
      <td>2019</td>
      <td>278</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2
points: 2
-->

In [15]:
query_q2 = """
SELECT startYear, COUNT(*) as total
FROM Title
WHERE startYear IS NOT NULL
GROUP BY startYear
ORDER BY startYear DESC
LIMIT 10
"""


res_q2 = pd.read_sql(query_q2, engine)
res_q2.head()

,startYear,total
0,2021,128
1,2020,181
2,2019,278
3,2018,282
4,2017,284


In [ ]:
grader.check("q2")

## Question 3: Report number of "Big-Hit" and "Not-Big-Hit" films.

Details: A "big hit" is a film with over 100,000 votes.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>

- Tests: Case, one-to-many Join
- Our solution is 10 lines.
    
- **Approach**: You need to (1) link films to ratings, (2) compute whether or not a film is a "hit", according to its ratings, and (3) group by and aggregate within hits and not hits.
    
- **Hint Data**: Use `Rating.numVotes` for number of votes.
- **Hint #1**: Use `JOIN`. See 9.4 @ 3:23 for an example https://youtu.be/FAYKJT1TFUA?t=199
- **Hint #2**: Use `CASE`. See 9.7 @ 0:40 for a brief overview https://youtu.be/6vkZUwkoAwM?t=40
- Review #3: For using `GROUP BY`, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166
   
</details>

Output whether or not the film is a hit `isBigHit` and number of films in that category `total`. Your output should match the following, exactly:

|isBigHit|total|
|---|---|
|no|5341|
|yes|2233|

<!--
BEGIN QUESTION
name: q3
points: 3
-->

In [16]:

query_q3 = """
SELECT 
CASE 
    WHEN CAST(numVotes as int) > 100000 THEN 'yes'
    ELSE 'no'
END AS 'isBigHit',
COUNT(*) AS total
FROM Rating
JOIN Title ON Rating.tconst = Title.tconst
GROUP BY isBigHit



"""


res_q3 = pd.read_sql(query_q3, engine)
res_q3

,isBigHit,total
0,no,5341
1,yes,2233


In [ ]:
grader.check("q3")

## Question 4: Compute average rating across films, per year.

**Warning**: `Rating` provides average rating *per film*, but we need average rating *across votes* per year. See hints for details.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>
    
    
- Tests: SQL aggregates + arithmetic
- Our solution is 7 lines
    
- **Approach**: You'll need to (1) link titles with ratings, (2) group by and aggregate within each title's release year, according to the formula above.
    

- **Hint Data**: Use `Title.startYear` for film release year.
- **Hint Data**: You shouldn't use the `AVG(...)` aggregate, because the `Rating` table contains *average ratings per film* and the number of votes. We want the *average across all votes*, for a year of films, which we can compute using: `sumRatingsPerYear / numVotesInYear = sum(averageFilmRating * numFilmVotes) / sum(numFilmVotes)` (the sums are across all films in a single year).
    
- Review #1: Use a `JOIN`. See 9.4 @ 3:23 for an example https://youtu.be/FAYKJT1TFUA?t=199
- Review #2: For using `GROUP BY`, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166. 
</details>

Output the year `year` and average rating `rating`, ordered by `year` with the most recent year at the top. The first 3 lines of your output should look like the following (but you should compute the first 10 rows):

|year|rating|
|---|---|
|2021|7.036896|
|2020|7.004193|
|2019|7.333072|

<!--
BEGIN QUESTION
name: q4
points: 3
-->

In [17]:

query_q4 = """
SELECT Title.startYear AS year, sum(Rating.averageRating * Rating.numVotes) / sum(Rating.numVotes) as rating
FROM Title
JOIN Rating on Title.tconst = Rating.tconst
GROUP BY startYear
ORDER BY startYear desc
LIMIT 10
"""


res_q4 = pd.read_sql(query_q4, engine)
res_q4.head()

,year,rating
0,2021,7.036896
1,2020,7.004193
2,2019,7.333072
3,2018,7.083549
4,2017,7.196795


In [ ]:
grader.check("q4")

## Question 5: Find people with the most ratings.

Details: Sort from highest to lowest.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>
    
- Tests: many-to-many join
- Our solution is 13 lines, excluding the `VIEW` scaffolding.
- **Approach**: You'll need to (1) link name to ratings, (2) group all ratings per person, and (3) compute total number of votes.
- **Hint #1**: Use multiple `JOIN`s. Here is an example with multiple JOINs (but you may want to watch the entire lesson from the start) 9.7 @ 5:32 https://youtu.be/6vkZUwkoAwM?t=332
- Review #2: For using GROUP BY, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166.
</details>

Output the person's `name`, average rating `rating`, and number of votes cast `numVotes`. The first 3 lines of your output should look like the following:

|nconst|name|numVotes|
|---|---|---|
|2369|John Williams|16574610|
|93|Brad Pitt|16007625|
|138|Leonardo DiCaprio|14757121|

<!--
BEGIN QUESTION
name: q5
points: 2
-->

In [18]:
query_q5 = """
CREATE VIEW PeopleRating AS
    SELECT Name.nconst, Name.primaryName AS name, sum(Rating.numVotes) AS numVotes
    FROM Name
    JOIN Role 
        ON Name.nconst = Role.nconst
    JOIN Rating
         ON Role.tconst = Rating.tconst
    GROUP BY primaryName
    ORDER BY numVotes desc
"""


#writing PeopleRating to db
engine.execute("DROP VIEW IF EXISTS PeopleRating")
engine.execute(query_q5)

res_q5 = pd.read_sql("SELECT * FROM PeopleRating LIMIT 10", engine)
res_q5


,nconst,name,numVotes
0,2369,John Williams,16574610
1,93,Brad Pitt,16007625
2,138,Leonardo DiCaprio,14757121
3,158,Tom Hanks,14703255
4,498278,Stan Lee,13906803
5,634240,Christopher Nolan,13728818
6,229,Steven Spielberg,13610887
7,134,Robert De Niro,13415268
8,168,Samuel L. Jackson,11997152
9,288,Christian Bale,11892101


In [ ]:
grader.check("q5")


## Question 6: Compute number of A/B/C-list people per film.

Details: A-list people have 1,000,000+ votes. B-list have 500,000 - 999,999 votes. C-list have 100,000 - 499,999. Return top 10 films, ranked by number of A-list actors/actresses.

Before looking below, take a moment to consider how you would do this problem. Would you use a `CASE`? (Why can't you?)

### Question 6a: Compute "X-List" classification per person.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>
    
- Tests: `AND` for predicates
- The solution is 7 lines.
</details>

Using `PeopleRating` from Question 5, compute booleans `isAList`, `isBList`, and `isCList` for each person. Additionally output `PeopleRating.nconst`. The first 3 lines of your output should look like the following (but when using this query for Question 6, compute the full table).

|isAList|isBList|isCList|nconst|
|---|---|---|---|
|1|0|0|2369|
|1|0|0|93|
|1|0|0|138|

<!--
BEGIN QUESTION
name: q6a
points: 2
-->

In [19]:

query_q6a = """
CREATE VIEW PeopleABC AS
    SELECT 
    CASE
        WHEN numVotes >= 1000000 THEN 1
        ELSE 0
    END AS 'isAList',
    CASE
        WHEN numVotes <= 999999 AND numVotes >= 500000 THEN 1
        ELSE 0
    END AS 'isBList',    
    CASE
        WHEN numVotes <= 499999 AND numVotes >= 100000 THEN 1
        ELSE 0
    END AS 'isCList',  
    nconst 
    FROM PeopleRating
    
"""

    
#writing PeopleABC to db
engine.execute("DROP VIEW IF EXISTS PeopleABC")
engine.execute(query_q6a)

res_q6a = pd.read_sql("SELECT * FROM PeopleABC LIMIT 10", engine)
res_q6a

,isAList,isBList,isCList,nconst
0,1,0,0,2369
1,1,0,0,93
2,1,0,0,138
3,1,0,0,158
4,1,0,0,498278
5,1,0,0,634240
6,1,0,0,229
7,1,0,0,134
8,1,0,0,168
9,1,0,0,288


In [ ]:
grader.check("q6a")

### Question 6b: Count up "X-List" people per film

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>
    
- Tests: When NOT to use case
    
- **Approach**: You need to: (1) Use a subquery, common table expression, or view to compute a boolean column for each "rank" -- `isAList`, `isBList` and `isCList` for every person; (2) link people ranks with titles; (3) group by and sum within each movie.
    
- **Hint #1**: Try Question 6a if this is confusing.
- Reivew #2: Use multiple JOINs. Here is an example with multiple JOINs (but you may want to watch the entire lesson from the start) 9.7 @ 5:32 https://youtu.be/6vkZUwkoAwM?t=332
- Review #3: For using GROUP BY, see lecture 8.6 @ 2:46 https://youtu.be/WsyA75ppEAU?t=166.
</details>

Output the number of A-list `numAList`, B-list `numBList`, and C-list `numCList` people, along with the film title. The first 3 lines of your output should look like the following (but you should compute the first 10 rows):

|numAList|	numBList|	numCList|	primaryTitle|
|---|---|---|---|
|10|	0|	0|	The Godfather |
|10|	0|	0|	The Godfather: Part II |
|10|	0|	0|	Star Wars: Episode V - The Empire Strikes Back |

<!--
BEGIN QUESTION
name: q6b
points: 3
-->

In [20]:
query_q6b = """
SELECT sum(PeopleABC.isAlist) AS numAList, sum(PeopleABC.isBlist) AS numBList, sum(PeopleABC.isClist) AS numCList, Title.primaryTitle
FROM PeopleABC
JOIN Role
ON PeopleABC.nconst = Role.nconst
JOIN Title
ON Role.tconst = Title.tconst
GROUP BY Title.tconst
ORDER BY numAList desc
LIMIT 10
"""


res_q6b = pd.read_sql(query_q6b, engine)
res_q6b

,numAList,numBList,numCList,primaryTitle
0,10,0,0,The Godfather
1,10,0,0,The Godfather: Part II
2,10,0,0,Star Wars: Episode V - The Empire Strikes Back
3,10,0,0,Back to the Future
4,10,0,0,Die Hard
5,10,0,0,Goodfellas
6,10,0,0,The Silence of the Lambs
7,10,0,0,Terminator 2: Judgment Day
8,10,0,0,A Few Good Men
9,10,0,0,Forrest Gump


In [ ]:
grader.check("q6b")

## Question 7: Find pairs of people that have 50+ collaborations.

Before looking below, take a moment to consider how you would do this problem. Can you do this without subqueries or common table expressions? Why not?

### Question 7a: For each movie, list all pairs of collaborators.

**Warning**: Make sure not to double-count pairs or to match any person with themselves as a "pair". See hints for details.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>

- Testing: self-joins with pairs

- Our solution has 8 lines excluding the `VIEW` scaffolding.
    
- **Approach**: You will need to (1) consider pairs and (2) filter pairs to avoid double-counting and/or counting one person with themselves as a pair.
    
- **Hint #1**: Pick whichever approach is more intuitive. You can (a) compute all pairs, then filter out invalid pairs using hint #2 like 9.4 @ 4:20 https://youtu.be/FAYKJT1TFUA?t=254 OR (b) write the more efficient inner JOIN, using hint #2 as the join predicate, like 9.4 @ 3:19 https://youtu.be/FAYKJT1TFUA?t=199
- **Hint #2**: To avoid double-counting and avoid counting one person with themselves as a pair, see 9.7 @ 7:19 https://youtu.be/6vkZUwkoAwM?t=439 
</details>

Output primary keys for both people `nconst1`, `nconst2` and for the film `tconst`. The first 3 lines of your output should look like the following (but you should compute the first 10 rows)

|nconst1	|nconst2	|tconst|
|---|---|---|
|617588	|29244	|417|
|617588	|76933	|417|
|617588	|242155	|417|

<!--
BEGIN QUESTION
name: q7a
points: 2
-->

In [21]:
query_q7a = """
CREATE VIEW Pair AS
    SELECT A.nconst as nconst1, B.nconst as nconst2, A.tconst
    FROM Role AS A
    JOIN Role AS B
    ON A.nconst > B.nconst
    AND A.tconst = B.tconst
    
    """


#writing Pair to db
engine.execute("DROP VIEW IF EXISTS Pair")
engine.execute(query_q7a)

res_q7a = pd.read_sql("SELECT * FROM Pair LIMIT 10", engine)
res_q7a.head()

,nconst1,nconst2,tconst
0,617588,29244,417
1,617588,76933,417
2,617588,242155,417
3,617588,324073,417
4,76933,29244,417


In [ ]:
grader.check("q7a")

### Question 7b: Report number of movies per pair.

Details: For all pairs that have collaborated at least 50 times.

<br/>

<details>
    <summary>▶ [Click to Expand] <b>Hints</b></summary>

- Tests: group by over multiple columns
- Our solution has 12 lines.
</details>

Output the names of both people, `name1` and `name2`, in addition to the number of collaborations `total`. The first 3 lines of your output should look like the following (but you should compute the whole table)

|name1|	name2|	total|
|---|---|---|
|D.B. Weiss	|David Benioff|	74|
|David Benioff|	George R.R. Martin|	73|
|D.B. Weiss|	George R.R. Martin|	73|

<!--
BEGIN QUESTION
name: q7b
points: 3
-->

In [22]:

query_q7b = """
SELECT  first.primaryName AS name1, second.primaryName AS name2, COUNT(*) as total
FROM Pair
JOIN Name AS first
ON Pair.nconst1 = first.nconst
JOIN Name AS second
ON Pair.nconst2 = second.nconst
GROUP BY nconst1, nconst2
HAVING total >= 50
ORDER BY total desc

"""


res_q7b = pd.read_sql(query_q7b, engine)
res_q7b.head()

,name1,name2,total
0,D.B. Weiss,David Benioff,74
1,David Benioff,George R.R. Martin,73
2,D.B. Weiss,George R.R. Martin,73
3,David Benioff,Peter Dinklage,59
4,D.B. Weiss,Peter Dinklage,59


In [ ]:
grader.check("q7b")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()